In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from torchvision.transforms import Resize, InterpolationMode, ToPILImage
from torchmetrics import JaccardIndex, Precision, Recall, F1Score
import segmentation_models_pytorch as smp

from src.models.BaselineModel import BaselineModel
from src.evaluation.evaluate_result import evaluate_result
from src.callbacks.SaveRandomImagesCallback import SaveRandomImagesCallback
from src.datasets.INRIAAerialImageLabellingDataset import (
    INRIAAerialImageLabellingDataset,
)

from src.datasets.utils.ResizeToDivisibleBy32 import ResizeToDivisibleBy32

d:\__repos\aerial_segmentation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prepare environment

In [2]:
torch.cuda.is_available()

True

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if you get some cryptic CUDA error, set device to "cpu" and try again
print(device)

cuda


In [4]:
VAL_SIZE = 0.2
BATCH_SIZE = 4
SEED = 42
IMAGE_SIZE = 576
SAVE_DIR = "outputs"
INRIA_DATASET_PATH = "data/INRIAAerialImageLabellingDataset"  # home PC
# INRIA_DATASET_PATH = "data/TestSubsets/INRIAAerialImageLabellingDataset"  # laptop

In [5]:
labeled_dataset = INRIAAerialImageLabellingDataset(
    INRIA_DATASET_PATH,
    split="train",
    transforms=[
        # Resize(IMAGE_SIZE),
        Resize(IMAGE_SIZE, interpolation=InterpolationMode.NEAREST_EXACT),
        # ResizeToDivisibleBy32()
    ],
)
print(len(labeled_dataset))

data/INRIAAerialImageLabellingDataset\train
180


In [6]:
test_dataset = INRIAAerialImageLabellingDataset(
    INRIA_DATASET_PATH,
    split="test",
    transforms=[
        # Resize(IMAGE_SIZE),
        Resize(IMAGE_SIZE, interpolation=InterpolationMode.NEAREST_EXACT),
        # ResizeToDivisibleBy32()
    ],
)
print(len(test_dataset))

data/INRIAAerialImageLabellingDataset\test
144


## Sanity check data

In [7]:
train_size = int(0.8 * len(labeled_dataset))
val_size = len(labeled_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(labeled_dataset, [train_size, val_size])

In [8]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [9]:
for images, masks in train_loader:
    print(images.shape)
    print(masks.shape)
    break

torch.Size([4, 3, 576, 576])
torch.Size([4, 1, 576, 576])


In [10]:
to_pil_transform = ToPILImage()

In [11]:
if BATCH_SIZE == 1:
    img = to_pil_transform(images.squeeze())

In [12]:
# img.show()

In [13]:
if BATCH_SIZE == 1:
    msk = to_pil_transform(masks.squeeze())

In [14]:
# msk.show()

# Training module

In [15]:
class SegmentationModel(pl.LightningModule):
    def __init__(self, model, learning_rate=1e-3):
        super(SegmentationModel, self).__init__()
        self.model = model
        self.learning_rate = learning_rate
        self.criterion = torch.nn.BCELoss()
        # self.criterion = smp.losses.JaccardLoss(mode="binary", from_logits=False)

        # Metrics
        self.train_iou = JaccardIndex(num_classes=2, task="binary")
        self.val_iou = JaccardIndex(num_classes=2, task="binary")
        self.train_precision = Precision(num_classes=2, task="binary")
        self.val_precision = Precision(num_classes=2, task="binary")
        self.train_recall = Recall(num_classes=2, task="binary")
        self.val_recall = Recall(num_classes=2, task="binary")
        self.train_f1 = F1Score(num_classes=2, task="binary")
        self.val_f1 = F1Score(num_classes=2, task="binary")

    def forward(self, x):
        x = self.model(x.to(device))
        output = (x > 0.5).float()
        return output

    def training_step(self, batch, batch_idx):
        images, masks = batch
        masks = torch.div(masks, 255).float()
        preds = self(images)
        loss = self.criterion(preds, masks)
        loss.requires_grad = True
        
        self.log('train_loss', loss, on_epoch=True)
        self.log('train_iou', self.train_iou(preds, masks), on_epoch=True)
        self.log('train_precision', self.train_precision(preds, masks), on_epoch=True)
        self.log('train_recall', self.train_recall(preds, masks), on_epoch=True)
        self.log('train_f1', self.train_f1(preds, masks), on_epoch=True)
        
        return loss

    def validation_step(self, batch, batch_idx):
        images, masks = batch
        masks = torch.div(masks, 255).float()
        preds = self(images)
        loss = self.criterion(preds, masks)
        loss.requires_grad = True
        
        self.log('val_loss', loss, on_epoch=True)
        self.log('val_iou', self.val_iou(preds, masks), on_epoch=True)
        self.log('val_precision', self.val_precision(preds, masks), on_epoch=True)
        self.log('val_recall', self.val_recall(preds, masks), on_epoch=True)
        self.log('val_f1', self.val_f1(preds, masks), on_epoch=True)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        images = batch
        preds = self(images)
        for i, pred in enumerate(preds):
            pred_img = to_pil_transform(pred.squeeze())
            pred_img.save(f"pred_{i}.png")
        

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [16]:
model = smp.Unet(
    encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,  # model output channels (number of classes in your dataset)
# ).to(device)
)

In [17]:
# model = smp.UnetPlusPlus(
#     encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=1,  # model output channels (number of classes in your dataset)
# ).to(device)

In [18]:
# model = smp.DeepLabV3(
#     encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=1,
# ).to(device)

In [19]:
# model = smp.DeepLabV3Plus(
#     encoder_name="resnet18",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
#     encoder_weights="imagenet",  # use `imagenet` pre-trained weights for encoder initialization
#     in_channels=3,  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
#     classes=1,
# ).to(device)

In [20]:
segmentation_model = SegmentationModel(model)

In [21]:
model_checkpoint_callback = ModelCheckpoint(
    monitor='val_loss', save_top_k=-1, mode='min'
    )

In [22]:
logger = CSVLogger("lightning_logs", name="inria_segmentation_model")

In [23]:
save_callback = SaveRandomImagesCallback(save_dir=SAVE_DIR)

In [24]:
trainer = pl.Trainer(
    max_epochs=10, 
    callbacks=[model_checkpoint_callback, save_callback], 
    logger=logger
    )

# DEBUG
# trainer = pl.Trainer(
#     max_epochs=1, 
#     callbacks=[model_checkpoint_callback, save_callback], 
#     logger=logger, 
#     accelerator="cpu"
#     )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [25]:
trainer.fit(segmentation_model, train_dataloaders=train_loader, val_dataloaders=val_loader)
 

You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type               | Params
-------------------------------------------------------
0 | model           | Unet               | 14.3 M
1 | criterion       | BCELoss            | 0     
2 | train_iou       | BinaryJaccardIndex | 0     
3 | val_iou         | BinaryJaccardIndex | 0     
4 | train_precision | BinaryPrecision    | 0     
5 | val_precision   | BinaryPrecision    | 0     
6 | train_recall    | BinaryRecall       | 0     
7 | val_recall      | BinaryRecall       | 0     
8 | train_f1        | BinaryF1Score      | 0     
9 | val_f1          | BinaryF1Score  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

d:\__repos\aerial_segmentation\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


d:\__repos\aerial_segmentation\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.
d:\__repos\aerial_segmentation\.venv\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:298: The number of training batches (36) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|██████████| 36/36 [01:29<00:00,  0.40it/s, v_num=20]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 36/36 [01:29<00:00,  0.40it/s, v_num=20]


In [26]:
trainer.test(ckpt_path="best", dataloaders=test_loader)

Restoring states from the checkpoint path at lightning_logs\inria_segmentation_model\version_20\checkpoints\epoch=0-step=36.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at lightning_logs\inria_segmentation_model\version_20\checkpoints\epoch=0-step=36.ckpt
d:\__repos\aerial_segmentation\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing DataLoader 0: 100%|██████████| 36/36 [00:47<00:00,  0.75it/s]


[{}]